In [1]:
!pip install -q --upgrade crewai openai langchain langchain-community langchain-openai gradio

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 42.8/42.8 kB 2.6 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 67.3/67.3 kB 4.5 MB/s eta 0:00:00
  Installing build dependencies ... done
  Getting requirements to build wheel ... done
  Preparing metadata (pyproject.toml) ... done
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 48.5/48.5 kB 3.1 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 365.3/365.3 kB 18.7 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 8.3/8.3 MB 104.1 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 16.4/16.4 MB 87.4 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 2.5/2.5 MB 67.3 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 70.6/70.6 kB 5.1 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 59.6/59.6 MB 10.0 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 324.5/324.5 kB 15.3 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 19.5/19.5 MB 82.1 MB/s eta 0

In [2]:
import gradio as gr
import os
import getpass
from langchain_openai import ChatOpenAI
from crewai import Agent, Task, Crew

In [3]:
if "OPENAI_API_KEY" not in os.environ or not os.environ["OPENAI_API_KEY"]:
    os.environ["OPENAI_API_KEY"] = getpass.getpass("🔐 Enter your OpenAI API key: ")

🔐 Enter your OpenAI API key: ··········


In [21]:
email_agent = Agent(
    role='HR Email Writer',
    goal='Compose formal, polite, context-aware emails for HR situations',
    backstory='Expert in professional communications and human resources',
    verbose=True,
    allow_delegation=False
)

In [22]:
blog_agent = Agent(
    role='Content Blogger',
    goal='Write engaging and informative blog posts based on user topics',
    backstory='Experienced blog writer skilled in multiple domains and styles',
    verbose=True,
    allow_delegation=False
)

In [23]:
summary_agent = Agent(
    role='Summarizer',
    goal='Create concise summaries from long pieces of text',
    backstory='Expert in text analysis and summarization',
    verbose=True,
    allow_delegation=False
)

In [24]:
qa_agent = Agent(
    role='Q&A Assistant',
    goal='Answer user queries in clear and concise language',
    backstory='Conversational expert trained on diverse questions and answers',
    verbose=True,
    allow_delegation=False
)

In [25]:
creative_agent = Agent(
    role='Creative Writer',
    goal='Generate poems, stories, or imaginative responses from prompts',
    backstory='Creative mind capable of generating interesting fictional content',
    verbose=True,
    allow_delegation=False
)

In [28]:
def run_crew(task_type, input_text):
    try:
        if task_type == "Email":
            task = Task(
                description=f"Compose a formal email: {input_text}",
                expected_output="A professional email suitable for HR communication.",
                agent=email_agent
            )
        elif task_type == "Blog Post":
            task = Task(
                description=f"Write a detailed blog post on: {input_text}",
                expected_output="An engaging and informative blog post.",
                agent=blog_agent
            )
        elif task_type == "Summary":
            task = Task(
                description=f"Summarize this text: {input_text}",
                expected_output="A concise and accurate summary.",
                agent=summary_agent
            )
        elif task_type == "Q&A":
            task = Task(
                description=f"Answer this question: {input_text}",
                expected_output="A clear and informative answer.",
                agent=qa_agent
            )
        elif task_type == "Creative Writing":
            task = Task(
                description=f"Write a story or poem about: {input_text}",
                expected_output="An imaginative and well-written piece of creative writing.",
                agent=creative_agent
            )
        else:
            return "⚠️ Invalid task type."

        crew = Crew(tasks=[task], agents=[task.agent], verbose=False)
        result = crew.kickoff()
        return result
    except Exception as e:
        return f"❌ An error occurred during generation:\n{str(e)}"

In [29]:
iface = gr.Interface(
    fn=run_crew,
    inputs=[
        gr.Dropdown([
            "Blog Post", "Email", "Summary", "Q&A", "Creative Writing"
        ], label="Task Type"),
        gr.Textbox(lines=4, placeholder="Enter your topic, question, or text here", label="Input")
    ],
    outputs=gr.Textbox(label="Generated Output"),
    title="🧠 GenAI Content Assistant (CrewAI powered)",
    description="An AI assistant powered by CrewAI that generates blog posts, emails, summaries, Q&A answers, or creative writing with expert agents."
)

try:
    iface.launch(share=True)
except Exception as e:
    print("❌ Gradio failed to launch. Here's the error:")
    print(e)

Colab notebook detected. To show errors in colab notebook, set debug=True in launch()
* Running on public URL: https://62d8e01c46cfa5cdcf.gradio.live

This share link expires in 1 week. For free permanent hosting and GPU upgrades, run `gradio deploy` from the terminal in the working directory to deploy to Hugging Face Spaces (https://huggingface.co/spaces)
